In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Refeence format
https://cookbook.openai.com/examples/how_to_format_inputs_to_chatgpt_models

# alpaca-cleaned

In [ ]:
# Expected dataset have openai-like messages format

dataset = load_dataset('yahma/alpaca-cleaned', split='train')

one_row = dataset[0]
def get_messages(one_row):
    # return openai messages format
    # Expected is a list of dictionaries with the keys "role" and "content", roles are "user" and "assistant" or system, note that gemma models does not support system role
    messages = [
        {"role": "user", "content": one_row["instruction"] + one_row["input"]},
        {"role": "assistant", "content": one_row["output"]},
    ]
    
    return {"messages": messages}

# Apply the chat template to each example in the dataset
dataset = dataset.map(get_messages)
dataset.save_to_disk('../data/alpaca-cleaned')
dataset

Saving the dataset (0/1 shards):   0%|          | 0/51760 [00:00<?, ? examples/s]

Dataset({
    features: ['output', 'input', 'instruction', 'messages'],
    num_rows: 51760
})

: 

# OpenO1-SFT

In [3]:
dataset = "O1-OPEN/OpenO1-SFT"
# Expected dataset have openai-like messages format

dataset = load_dataset(dataset, split='train')
one_row = dataset[0]
print(one_row)

{'instruction': "Consider a regular octagon. How many different triangles can be formed if the octagon is placed inside a circle and we can also use the center of the circle as a vertex for the triangles? Let's think step by step.", 'output': "<Thought>\nAlright, I need to figure out how many different triangles can be formed in a regular octagon that's placed inside a circle, with the added twist that the center of the circle can also be used as a vertex for the triangles. Let's break this down step by step.\n\nFirst, let's recall that a regular octagon has 8 equal sides and 8 equal angles. Since it's placed inside a circle, each vertex of the octagon lies on the circumference of the circle. Additionally, we have the center of the circle as an extra point that can be used as a vertex for forming triangles.\n\nSo, in total, we have 9 points to consider: the 8 vertices of the octagon and the center of the circle.\n\nNow, to form a triangle, we need to select any 3 of these 9 points. The

In [4]:



def get_messages(one_row):
    # return openai messages format
    # Expected is a list of dictionaries with the keys "role" and "content", roles are "user" and "assistant" or system, note that gemma models does not support system role
    messages = [
        {"role": "user", "content": one_row["instruction"]},
        {"role": "assistant", "content": one_row["output"]},
    ]
    
    return {"messages": messages}

# Apply the chat template to each example in the dataset
dataset = dataset.map(get_messages)
dataset.save_to_disk('../data/OpenO1-SFT')
dataset

Map:   0%|          | 0/77685 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/77685 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'output', 'messages'],
    num_rows: 77685
})